<a href="https://colab.research.google.com/github/canekkazan/harvard-orchid-flowers-species-classification/blob/main/classify_orchid_species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


In [ ]:
!unzip '/content/drive/MyDrive/Datasets/Orchid Flowers Dataset-v1.1.zip'

In [ ]:
train_df = pd.read_csv('/content/Orchid Flowers Dataset-v1.1/Species_Classifier/Training_Data_for_Species.txt',header = None,dtype=str)
valid_df = pd.read_csv('/content/Orchid Flowers Dataset-v1.1/Species_Classifier/Validation_Data_for_Species.txt',header = None,dtype=str)
test_df = pd.read_csv('/content/Orchid Flowers Dataset-v1.1/Species_Classifier/Testing_Data_for_Species.txt',header = None,dtype=str)


columns = ['Image_name', 'Species']
train_df.columns = columns
valid_df.columns = columns
test_df.columns = columns

In [ ]:
# Augment with rotated, shifted, flipped, images

train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

data_gen = train_datagen = ImageDataGenerator(
    rescale=1./255,
)   

In [ ]:
batch_size = 64

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
dataframe=train_df,
directory="Orchid_Images",
x_col="Image_name",
y_col="Species",
subset=None,
batch_size=batch_size,
seed=123,
shuffle=True,
class_mode="sparse",
save_format="png",
target_size=(32,32),
validate_filenames=False)

valid_generator=data_gen.flow_from_dataframe(
dataframe=valid_df,
directory="Orchid_Images",
x_col="Image_name",
y_col="Species",
subset=None,
batch_size=batch_size,
seed=123,
shuffle=True,
save_format="png",
class_mode="sparse",
target_size=(32,32),
validate_filenames=False)

test_generator=data_gen.flow_from_dataframe(
dataframe=test_df,
directory="Orchid_Images",
x_col="Image_name",
y_col="Species",
subset=None,
batch_size=batch_size,
seed=123,
shuffle=True,
save_format="png",
class_mode="sparse",
target_size=(32,32),
validate_filenames=False)


Found 5119 non-validated image filenames belonging to 156 classes.
Found 1235 non-validated image filenames belonging to 156 classes.
Found 802 non-validated image filenames belonging to 156 classes.


In [ ]:
#Define model
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(64,(5, 5)))
model.add(Activation('relu'))

model.add(Conv2D(32, (7, 7), padding='same'))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(156))
model.add(Activation('softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 activation_6 (Activation)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        102464    
                                                                 
 activation_7 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        100384    
                                                                 
 activation_8 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 25088)            

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50
)